In [ ]:
!pip install pinecone-client==3.2.2 sentence-transformers==2.7.0 datasets==2.19.0 transformers==4.40.1 openai==1.25.2 llama-index==0.10.34 llama-index-vector-stores-pinecone==0.1.6  -qqq
# faiss-cpu==1.7.2
!pip install -q condacolab
import condacolab
condacolab.install()

!conda install -c pytorch faiss-cpu -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 141.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.

# **실습 데이터 다운로드**

In [ ]:
!wget ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz
!tar -xf sift.tar.gz
!mkdir data/sift1M -p
!mv sift/* data/sift1M

--2025-03-18 09:36:32--  ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz
           => ‘sift.tar.gz.2’
Resolving ftp.irisa.fr (ftp.irisa.fr)... 131.254.254.45, 2001:660:7303:254::45
Connecting to ftp.irisa.fr (ftp.irisa.fr)|131.254.254.45|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /local/texmex/corpus ... done.
==> SIZE sift.tar.gz ... 168280445
==> PASV ... done.    ==> RETR sift.tar.gz ... done.
Length: 168280445 (160M) (unauthoritative)

sift.tar.gz.2       100%[===================>] 160.48M  2.79MB/s    in 37s     

2025-03-18 09:37:11 (4.33 MB/s) - ‘sift.tar.gz.2’ saved [168280445]



# **실습 데이터 불러오기**

In [ ]:
import psutil

def get_memory_usage_mb():
  # 현재 실행 중인 프로세스 정보 가져오기
  process = psutil.Process()
  # 프로세스의 메모리 사용량 정보 가져오기
  memory_info = process.memory_info()

  # memory_info.rss -> Resident Set Size(RSS) -> 프로세스가 실제로 물리적 메모리에 로드된 크기
  # 바이트를 MB로 변환
  return memory_info.rss / (1024 * 1024)

In [ ]:
import time
import faiss
from faiss.contrib.datasets import DatasetSIFT1M

# DatasetSIFT1M -> FAISS에서 제공하는 SIFT1M 데이터셋을 로드하는 클래스
# SIFT1M -> 100만 개(1M) SIFT(Scale-Invariant Feature Transform) 벡터로 구성된 데이터셋
# 이미지 검색, 유사도 검색 등의 벤치마킹에 많이 사용됨
ds = DatasetSIFT1M()

# 검색에 사용할 데이터
xq = ds.get_queries()
# 저장된 벡터 데이터
xb = ds.get_database()
# 질문에 대한 실제 정답 데이터
gt = ds.get_groundtruth()

# **데이터가 늘어날 때 색인/검색 시간, 메모리 사용량 변화**

In [ ]:
k = 1
# 벡터 차원 추출 (128)
d = xq.shape[1]
nq = 1000
xq = xq[:nq]

# i를 2씩 증가
for i in range(1, 10, 2):
  start_memory = get_memory_usage_mb()
  start_indexing = time.time()

  index = faiss.IndexFlatL2(d)
  index.add(xb[:(i + 1) * 100000])

  end_indexing = time.time()
  end_memory = get_memory_usage_mb()

  t0 = time.time()
  D, I = index.search(xq, k)
  t1 = time.time()

  print(f"데이터 {(i + 1) * 100000}개:")
  print(f"색인: {(end_indexing - start_indexing) * 1000 :.3f} ms ({end_memory - start_memory:.3f} MB) 검색: {(t1 - t0) * 1000 / nq :.3f} ms")

ValueError: input not a numpy array

# **파라미터 m의 변경에 따른 성능 확인**

In [ ]:
import numpy as np

k = 1
d = xq.shape[1]
nq = 1000
xq = xq[:nq]

# 강제 변환 (안전하게)
xb = np.ascontiguousarray(xb, dtype=np.float32)
print(f"xb type: {type(xb)}")
print(f"xb dtype: {xb.dtype}")
print(f"xb shape: {xb.shape}")
print(f"Is xb C-contiguous? {xb.flags['C_CONTIGUOUS']}")

# xq = np.ascontiguousarray(xq, dtype=np.float32)
# print(f"xq type: {type(xq)}")
# print(f"xq dtype: {xq.dtype}")
# print(f"xq shape: {xq.shape}")
# print(f"Is xq C-contiguous? {xq.flags['C_CONTIGUOUS']}")

for m in [8, 16, 32, 64]:
  index = faiss.IndexHNSWFlat(d, m)

  time.sleep(3)

  start_memory = get_memory_usage_mb()
  start_index = time.time()

  try:
    index.add(xb)
    print("FAISS IndexHNSWFlat 생성 및 xb 추가 성공!")
  except ValueError as e:
    print(f"IndexHNSWFlat 인덱스 추가 중 에러 발생: {e}")

  end_memory = get_memory_usage_mb()
  end_index = time.time()

  print(f"M: {m} - 색인 시간: {end_index - start_index} s, 메모리 사용량: {end_memory - start_memory} MB")

  t0 = time.time()
  D, I = index.search(xq, k)
  t1 = time.time()

  # gt[:nq, :1] -> gt[:1000, :1] -> 즉 추린 질의 1000개 만큼의 정답 데이터 1000개를 가져와 실제 정답 인덱스인 1을 추출
  # 정답 데이터를 모두 더해서 전체 쿼리 개수로 나눠 계산한다
  recall_at_1 = np.equal(I, gt[:nq, :1]).sum() / float(nq)
  print(f"{(t1 - t0) * 1000.0 / nq:.3f} ms per query, R@1 {recall_at_1:.3f}")

xb type: <class 'numpy.ndarray'>
xb dtype: float32
xb shape: (1000000, 128)
Is xb C-contiguous? True
FAISS IndexHNSWFlat 생성 및 xb 추가 성공!
M: 8 - 색인 시간: 111.28687405586243 s, 메모리 사용량: 752.109375 MB
0.040 ms per query, R@1 0.697
FAISS IndexHNSWFlat 생성 및 xb 추가 성공!
M: 16 - 색인 시간: 120.2768120765686 s, 메모리 사용량: 632.86328125 MB
0.051 ms per query, R@1 0.785
FAISS IndexHNSWFlat 생성 및 xb 추가 성공!
M: 32 - 색인 시간: 233.6597261428833 s, 메모리 사용량: 755.3828125 MB
0.097 ms per query, R@1 0.904
FAISS IndexHNSWFlat 생성 및 xb 추가 성공!
M: 64 - 색인 시간: 297.6750257015228 s, 메모리 사용량: 999.16015625 MB
0.220 ms per query, R@1 0.934


# **ef_construction을 변화시킬 때 성능 확인**

In [ ]:
k = 1
d = xq.shape[1]
nq = 1000
xq = xq[:nq]

for ef_construction in [40, 80, 160, 320]:
  index = faiss.IndexHNSWFlat(d, 32)
  index.hnsw.efConstruction = ef_construction

  time.sleep(3)

  start_memory = get_memory_usage_mb()
  start_index = time.time()

  index.add(xb)

  end_memory = get_memory_usage_mb()
  end_index = time.time()

  print(f"efConstruction: {ef_construction} - 색인 시간: {end_index - start_index} s, 메모리 사용량: {end_memory - start_memory} MB")



# **ef_search 변경에 따른 성능 확인**

In [ ]:
for ef_search in [16, 32, 64, 128]:
  index.hnsw.efSearch = ef_search

  t0 = time.time()

  D, I = index.search(xq, k)

  t1 = time.time()

  recall_at_1 = np.equal(I, gt[:nq, :1]).sum() / float(nq)

  print(f"{(t1 - t0) * 1000.0 / nq:.3f} ms per query, R@1 {recall_at_1:.3f}")

# **파인콘 계정 연결 및 인덱스 생성**

In [ ]:
# ServerlessSpec -> 서버리스 인덱스 설정
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = "자신의 API 키를 입력"
pc = Pinecone(api_key=pinecone_api_key)

# ServerlessSpec("aws", "us-east-1") -> Pinecone의 서버리스 모드에서 인덱스를 AWS의 us-east-1 리전에 생성
# 꼭 서버리스로 사용하지 않아도 된다
pc.create_index("llm-book", spec=ServerlessSpec("aws", "us-east-1"), dimension=768)
index = pc.index('llm-book')

# **임베딩 생성**

In [ ]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

# 임베딩 모델 불러오기
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 데이터셋 불러오기
klue_dp_train = load_dataset('klue', 'dp', split='train')

embeddings = sentence_model.encode(klue_dp_train['sentence'])

# **파인콘 입력을 위한 데이터 형태 변경**

In [ ]:
# 파이썬 기본 데이터 타입으로 변경
embeddings = embedding.tolist()

# {"id": 문서 ID(str), "values": 벡터 임베딩(List[float]), "metadata": 메타 데이터(dict) ) 형태로 데이터 준비
insert_data = []

for idx, (embedding, text) in enumerate(zip(embeddings, klue_dp_train['sentence'])):
  insert_data.append({"id": str(idx), "values": embedding, "metadata": {'text': text}})

# **임베딩 데이터를 인덱스에 저장**

In [ ]:
upsert_response = index.upsert(vectors = insert_data, namespace='llm-book-sub')

# **인덱스 검색하기**

In [ ]:
query_response = index.query(
    namespace='llm-book-sub', # 검색할 네임스페이스
    top_k=10, # 몇 개의 결과를 반환할지
    include_values=True, # 벡터 임베딩 반환 여부
    include_meatadata=True, # 메타 데이터 반환 여부
    vector=embeddings[0] # 검색할 벡터 임베딩
)

query_response

# **파인콘에서 문서 수정 및 삭제**

In [ ]:
new_text = '변경할 새로운 텍스트'
new_embedding = sentence_model.encode(new_text).tolist()

# 업데이트
update_response = index.update(
    id='기존_문서_id',
    values=new_embedding,
    set_metadata={'text': new_text},
    namespace='llm-book-sub'
)

# 삭제
delete_response = index.delete(ids=['기존_문서_id'], namespace='llm-book-sub')

# **라마인덱스에서 다른 벡터 데이터베이스 사용**

In [ ]:
# 파인콘 기본 설정
from pinecone import Pinecone, ServerlessSpec

# metric="euclidean" -> 유사도 측정 방식 / euclidean -> 유클리드 거리(가장 가까운 벡터)
pc = Pinecone(api_key=pinecone_api_key)
pc.create_index(
    "quickstart", dimension=1536, metric="euclidean", spec=ServerlessSpec("aws", "us-east-1")
)
pinecone_index = pc.index("quickstart")

# 라마인덱스에 파인콘 인덱스 연결
from llama_index.core import VectorStoreIndex
# 라마인덱스에서 파인콘을 사용하려면 필요
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import StorageContext

# 파인콘을 라마인덱스에서 사용하기 위해 라마인덱스에 맞게 래핑
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
# 라마인덱스 벡터 저장소 변경
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# 인덱스 생성, 저장(문서 리스트, 저장소)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

# **실습 데이터셋 다운로드**

In [ ]:
from datasets import load_dataset

dataset = load_dataset("poloclub/diffusiondb", "2m_first_1k", split="train")

example_index = 867
original_image = dataset[example_index]['image']
original_prompt = dataset[exmaple_index]['prompt']
print(original_prompt)

# **GPT-4o 요청에 사용할 함수**

In [ ]:
import requests
import base64
from io import BytesIO

def make_base64(image):
  buffered = BytesIO()
  # 이미지를 JPEG 형식으로 변환하여 메모리에 저장
  image.save(buffered, format="JPEG")
  # Base64 인코딩 후 문자열로 변환
  img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')

  return img_str

# gpt4에 이미지와 함께 요청 보내기
def generate_description_from_image_gpt4(prompt, image64):
  headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {client.api_key}"
  }

  payload = {
      "model": "gpt-4o",
      "messages": [
          {
              "role": "user",
              "content": [
                  {
                      "type": "text",
                      "text": prompt
                  },
                  {
                      "type": "image_url",
                      "image_url": {
                          "url": f"data:image/jpeg;base64,{image64}"
                      }
                  }
              ]
          }
      ],
      "max_tokens": 300
  }

  response_oai = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
  result = response_oai.json()['choices'][0]['message']['content']

  return result

# **이미지 설명 생성**

In [ ]:
image_base64 = make_base64(original_image)
described_result = generate_description_from_image_gpt4("Describe provided image", image_base64)

described_result
# The image depicts a digitally created, fantastical creature that combines features of different animals. It has the body and face of a lion, with a rich, golden mane that transitions into an array of vibrant, peacock-like feathers. The feathers themselves are full of brilliant colors, primarily blues and greens, with "eyes" that mimic the look of a peacock's plumage. The creature is sitting down and facing forward with a calm and majestic expression.
# The creature is set against a picturesque backdrop that resembles a lush, blooming meadow or garden, with rolling green hills in the distance and a blue sky above. The colors are rich and the composition is balanced, emphasizing the surreal and regal aspect of the creature. It's an imaginative piece that blends the natural elements of these animals in a mystical way.
# 이 이미지는 다양한 동물의 특징을 결합한 디지털로 창조된 환상적인 생물을 묘사합니다. 이 동물은 사자의 몸과 얼굴을 하고 있으며, 풍성한 황금빛 갈기가 공작새와 같은 생생한 깃털로 변합니다. 깃털은 주로 파란색과 녹색의 화려한 색상으로 가득하며, 공작의 깃털을 닮은 '눈'이 있습니다. 이 생물은 차분하고 장엄한 표정으로 앉아서 정면을 바라보고 있습니다.
# 이 생물은 무성하고 꽃이 만발한 초원이나 정원을 연상시키는 그림 같은 배경을 배경으로 멀리 푸른 언덕이 펼쳐져 있고 위로는 푸른 하늘이 펼쳐져 있습니다. 색상이 풍부하고 구도가 균형 잡혀 있어 초현실적이고 당당한 생물의 모습을 강조합니다. 동물의 자연적 요소를 신비로운 방식으로 혼합한 상상력이 돋보이는 작품입니다.

# **클라이언트 준비**

In [ ]:
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = pinecone_api_key # '자신의 파인콘 API 키 입력'
openai_api_key = '자신의 OpenAI API 키 입력'

pc = Pinecone(api_key=pinecone_api_key)
os.environ["OPENAI_API_KEY"] = openai_api_key
client = OpenAI()

# **인덱스 생성**

In [ ]:
print(pc.list_indexes())

index_name = "llm-multimodal"

try:
  pc.create_index(
      name=index_name,
      dimension=512,
      metric="cosine,
      spec=ServerlessSpec("aws", "us-east-1")
  )

  # pc.list_indexes() -> 파인콘에 현재 존재하는 인덱스 목록을 반환
  print(pc.list_indexes())
except:
  print("Index already exists")

index = pc.Index(index_name)

# **프롬프트 텍스트를 텍스트 임베딩 모델을 활용해 임베딩 벡터로 변환**

In [ ]:
import torch
# 반복문 진행률을 표시하는 tqdm 라이브러리
from tqdm.auto import trange
from torch.utils.data import DataLoader
# Hugging Face의 CLIP 모델 관련 클래스
from transformers import AutoTokenizer, CLIPTextModelWithProjection

device = "cuda" if torch.cuda.is_available() else "cpu"

# CLIP 모델과 토크나이저 로드
text_model = CLIPTextModelWithProjection.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

# 이미지 생성 프롬프트 토큰화
# padding=True -> 문장 길이 맞춤
# return_tensors="pt" -> 파이토치 텐서 형태로 변환
# truncation=True -> 너무 긴 문장은 자름
tokens = tokenizer(dataset['prompt'], padding=True, return_tensors="pt". truncation=True)
batch_size = 16
text_embs = []

# trange -> 진행바 표시
# 0 ~ dataset의 크기까지 batch_size 만큼 증가시키며 반복
for start_idx in trange(0, len(dataset), batch_size):
  # 학습이 아니라 추론이므로 그래디언트 계산 비활성화
  with torch.no_grad():
    # input_ids = tokens['input_ids'] -> 토크나이저가 변환한 문장의 토큰 ID
    # attention_mask = tokens['attention_mask'] -> 패딩된 부분을 무시하고 실제 단어만 처리하도록 도와주는 마스크

    # 문장을 임베딩으로 변환
    outputs = text_model(input_ids = tokens['input_ids'][start_idx:start_idx + batch_size],
                         attention_mask = tokens['attention_mask'][start_idx:start_idx + batch_size])

    # outputs는 모델의 결과 / outputs.text_embeds -> 그 중에서 텍스트 임베딩 벡터만 가져온다
    text_emb_tmp = outputs.text_embeds

  text_embs.append(text_emb_tmp)

# torch.cat(text_embs, dim=0): 리스트에 저장된 배치별 결과를 하나의 큰 텐서로 합침
text_embs = torch.cat(text_embs, dim=0)
text_embs.shape# (1000, 512)

# **텍스트 임베딩 벡터를 파인콘 인덱스에 저장**

In [ ]:
input_data = []

# id_int -> range(0, len(dataset))
# emb -> text_embs.tolist()
# prompt -> dataset['prompt']
for id_int, emb, prompt in zip(range(0, len(dataset)), text_embs.tolist(), dataset['prompt']):
  # id는 반복문 인덱스
  # values는 텍스트 임베딩
  # prompt는 요청 프롬프트 원본
  input_data.append(
      {
          "id": str(id_int),
          "values": emb,
          "metadata": {
              "prompt": prompt
          }
      }
  )

index.upsert(
    vectors=input_data
)